In [1]:
import sympy as sp

### Parameters and Variables

In [2]:
#Variables
x,T = sp.symbols('x,T', real=True, positive=True)
#Parameters
w = sp.IndexedBase('\Gamma', real=True, positive=True)
m = sp.IndexedBase('m', real=True, positive=True)
i,j,l,k = sp.symbols('i j l k', integer=True)
nDec, nF = sp.symbols('n_{dec}, n_{F}', integer=True)
nComp = sp.symbols('n_{comp}', integer=True)
g = sp.IndexedBase('g')
BR = sp.IndexedBase(r'BR')

### Functions

In [3]:
n = sp.IndexedBase(r'n')
neq = sp.IndexedBase(r'\bar{n}')
d = sp.IndexedBase(r'\delta')
D = sp.IndexedBase(r'\Delta')
Nth = sp.IndexedBase(r'\mathcal{N}^{th}')
N0th = sp.IndexedBase(r'\mathcal{N}^{0th}')
N1th = sp.IndexedBase(r'\mathcal{N}^{1th}')

In [4]:
#Functions
NS = sp.symbols(r'N_S', cls=sp.Symbol)
R = sp.IndexedBase(r'R')
Pn = sp.IndexedBase(r'P_{n}')
H = sp.symbols('H', cls=sp.Symbol)
sv = sp.IndexedBase(r'\langle\sigma\text{v}\rangle')
crate = sp.IndexedBase(r'\tilde{\Gamma}_{SM}')
Beff = sp.IndexedBase(r'\mathcal{B}^{eff}')

In [5]:
#Terms to be collected
coll = (crate[i,j],w[i],Beff[i,j],Beff[j,i],sv[j,j],sv[i,j],sv[i,i])

## Number Density Equations:

In [6]:
dNi = -3 
dNi += (n[i]/H)*(neq[i]**2/n[i]**2-1)*sv[i,i]
dNi += (1/H)*(neq[i]*neq[j]/n[i]-n[j])*sv[i,j]
dNi += (n[i]/H)*(neq[i]**2*n[j]**2/(n[i]**2*neq[j]**2)-1)*sv[j,j]
dNi += (1/H)*(neq[i]*n[j]/(n[i]*neq[j])-1)*crate[i,j]
dNi += -(w[i]*m[i]/(H*R[i]))*(1-Nth[i]/n[i])
dNi += Beff[j,i]*(w[j]*m[j]/(H*R[j]))*(n[j]/n[i] - Nth[j,i]/n[i])
dNi = sp.collect(dNi,coll)

In [7]:
sp.collect(dNi,coll)

-3 + (\bar{n}[i]**2/n[i]**2 - 1)*\langle\sigma\text{v}\rangle[i, i]*n[i]/H - (-\mathcal{N}^{th}[i]/n[i] + 1)*\Gamma[i]*m[i]/(H*R[i]) + (-\mathcal{N}^{th}[j, i]/n[i] + n[j]/n[i])*\Gamma[j]*\mathcal{B}^{eff}[j, i]*m[j]/(H*R[j]) + (\bar{n}[i]*\bar{n}[j]/n[i] - n[j])*\langle\sigma\text{v}\rangle[i, j]/H + (\bar{n}[i]*n[j]/(\bar{n}[j]*n[i]) - 1)*\tilde{\Gamma}_{SM}[i, j]/H + (\bar{n}[i]**2*n[j]**2/(\bar{n}[j]**2*n[i]**2) - 1)*\langle\sigma\text{v}\rangle[j, j]*n[i]/H

### Expansion of equations

In [8]:
#Replace n -> neq(1+delta) + Delta
#Coupled regime: Delta = 0, delta << 1 (expansion up to first order in delta)
#Non-coupled regime: delta = 0 (expansion up to order zero in delta)
#General scenario: expansion up to first order in delta and set delta_i*Delta_i = 0
dNiGeneral = dNi.subs({n[i] : neq[i]*(1+d[i])+D[i],n[j] : neq[j]*(1+d[j])+D[j]})
dNiGeneral = sp.collect(sp.expand(dNiGeneral).subs({d[i]*D[i] : 0, d[j]*D[j] : 0}),coll)

In [9]:
#Zero order term:
dNiGeneral0 = dNiGeneral.taylor_term(0,d[i]).taylor_term(0,d[j])
#First order term (1st order in delta_i with Delta_i = 0 + 1st order in delta_j with Delta_j=0)
dNiGeneral1 = dNiGeneral.taylor_term(1,d[i]).subs({D[i] : 0}).taylor_term(0,d[j])
dNiGeneral1 += dNiGeneral.taylor_term(0,d[i]).taylor_term(1,d[j]).subs({D[j] : 0})

In [10]:
exp0 = -3
for term in coll:
    exp0 += sp.simplify(dNiGeneral0.coeff(term,1))*term
exp0

-3 + (-(\Delta[i] + \bar{n}[i])*(\Delta[j] + \bar{n}[j]) + \bar{n}[i]*\bar{n}[j])*\langle\sigma\text{v}\rangle[i, j]/(H*(\Delta[i] + \bar{n}[i])) + (-\Delta[i]*\bar{n}[j] + \Delta[j]*\bar{n}[i])*\tilde{\Gamma}_{SM}[i, j]/(H*(\Delta[i] + \bar{n}[i])*\bar{n}[j]) - (\Delta[i] + 2*\bar{n}[i])*\Delta[i]*\langle\sigma\text{v}\rangle[i, i]/(H*(\Delta[i] + \bar{n}[i])) + (-\Delta[i] - \bar{n}[i] + \mathcal{N}^{th}[i])*\Gamma[i]*m[i]/(H*(\Delta[i] + \bar{n}[i])*R[i]) + (\Delta[j] + \bar{n}[j] - \mathcal{N}^{th}[j, i])*\Gamma[j]*\mathcal{B}^{eff}[j, i]*m[j]/(H*(\Delta[i] + \bar{n}[i])*R[j]) + (-\Delta[i]**2*\bar{n}[j]**2 - 2*\Delta[i]*\bar{n}[i]*\bar{n}[j]**2 + \Delta[j]**2*\bar{n}[i]**2 + 2*\Delta[j]*\bar{n}[i]**2*\bar{n}[j])*\langle\sigma\text{v}\rangle[j, j]/(H*(\Delta[i] + \bar{n}[i])*\bar{n}[j]**2)

In [11]:
sp.simplify(exp0-dNiGeneral0)

0

In [12]:
exp1 = 0
for dterm in (d[i],d[j]):
    for term in coll:
        exp1 += sp.simplify(dNiGeneral1.coeff(dterm,1).coeff(term))*term*dterm
sp.collect(exp1,(d[i],d[j]))

(-\bar{n}[j]*\langle\sigma\text{v}\rangle[i, j]/H + 2*\bar{n}[i]**2*\langle\sigma\text{v}\rangle[j, j]/(H*(\Delta[i] + \bar{n}[i])) + \bar{n}[i]*\tilde{\Gamma}_{SM}[i, j]/(H*(\Delta[i] + \bar{n}[i])) + \Gamma[j]*\bar{n}[j]*\mathcal{B}^{eff}[j, i]*m[j]/(H*(\Delta[i] + \bar{n}[i])*R[j]))*\delta[j] + (-(\Delta[j] + \bar{n}[j])*\tilde{\Gamma}_{SM}[i, j]/(H*\bar{n}[j]) + (-\Delta[j] - \bar{n}[j] + \mathcal{N}^{th}[j, i])*\Gamma[j]*\mathcal{B}^{eff}[j, i]*m[j]/(H*R[j]*\bar{n}[i]) - (\Delta[j]**2 + 2*\Delta[j]*\bar{n}[j] + 2*\bar{n}[j]**2)*\bar{n}[i]*\langle\sigma\text{v}\rangle[j, j]/(H*\bar{n}[j]**2) - 2*\bar{n}[i]*\langle\sigma\text{v}\rangle[i, i]/H - \bar{n}[j]*\langle\sigma\text{v}\rangle[i, j]/H - \Gamma[i]*\mathcal{N}^{th}[i]*m[i]/(H*R[i]*\bar{n}[i]))*\delta[i]

In [13]:
sp.simplify(exp1-dNiGeneral1)

0

### Check limits

#### Coupled Regime

In [14]:
dNiCoupled = (dNiGeneral0+ dNiGeneral1).replace(D[i],0).replace(d[j],0).replace(D[j],n[j]-neq[j])
sp.collect(sp.collect(sp.simplify(dNiCoupled),d[i]),coll)

(-3*H*R[i]*R[j]*\bar{n}[i]*\bar{n}[j]**2 - (\bar{n}[i] - \mathcal{N}^{th}[i])*R[j]*\Gamma[i]*\bar{n}[j]**2*m[i] - (\bar{n}[j] - n[j])*(\bar{n}[j] + n[j])*R[i]*R[j]*\bar{n}[i]**2*\langle\sigma\text{v}\rangle[j, j] + (\bar{n}[j] - n[j])*R[i]*R[j]*\bar{n}[i]*\bar{n}[j]**2*\langle\sigma\text{v}\rangle[i, j] - (\bar{n}[j] - n[j])*R[i]*R[j]*\bar{n}[i]*\bar{n}[j]*\tilde{\Gamma}_{SM}[i, j] - (\mathcal{N}^{th}[j, i] - n[j])*R[i]*\Gamma[j]*\bar{n}[j]**2*\mathcal{B}^{eff}[j, i]*m[j] + (-(2*\bar{n}[i]*\langle\sigma\text{v}\rangle[i, i] + \bar{n}[j]*\langle\sigma\text{v}\rangle[i, j])*R[i]*R[j]*\bar{n}[i]*\bar{n}[j]**2 - (-\mathcal{N}^{th}[j, i] + n[j])*R[i]*\Gamma[j]*\bar{n}[j]**2*\mathcal{B}^{eff}[j, i]*m[j] - ((\bar{n}[j] - n[j])**2 - 2*(\bar{n}[j] - n[j])*\bar{n}[j] + 2*\bar{n}[j]**2)*R[i]*R[j]*\bar{n}[i]**2*\langle\sigma\text{v}\rangle[j, j] - R[i]*R[j]*\bar{n}[i]*\bar{n}[j]*\tilde{\Gamma}_{SM}[i, j]*n[j] - R[j]*\Gamma[i]*\bar{n}[j]**2*\mathcal{N}^{th}[i]*m[i])*\delta[i])/(H*R[i]*R[j]*\bar{n}[

#### Doubly Coupled Regime

In [15]:
dNiCoupled = (dNiGeneral0+ dNiGeneral1).replace(D[i],0).replace(D[j],0)
sp.collect(sp.collect(dNiCoupled,(d[i],d[j])),coll)

(-m[i]/(H*R[i]) + \mathcal{N}^{th}[i]*m[i]/(H*R[i]*\bar{n}[i]))*\Gamma[i] + (\Gamma[j]*\bar{n}[j]*m[j]/(H*R[j]*\bar{n}[i]) - \Gamma[j]*\mathcal{N}^{th}[j, i]*m[j]/(H*R[j]*\bar{n}[i]))*\mathcal{B}^{eff}[j, i] + (2*\bar{n}[i]*\langle\sigma\text{v}\rangle[j, j]/H - \bar{n}[j]*\langle\sigma\text{v}\rangle[i, j]/H + \tilde{\Gamma}_{SM}[i, j]/H + \Gamma[j]*\bar{n}[j]*\mathcal{B}^{eff}[j, i]*m[j]/(H*R[j]*\bar{n}[i]))*\delta[j] + ((-\Gamma[j]*\bar{n}[j]*m[j]/(H*R[j]*\bar{n}[i]) + \Gamma[j]*\mathcal{N}^{th}[j, i]*m[j]/(H*R[j]*\bar{n}[i]))*\mathcal{B}^{eff}[j, i] - 2*\bar{n}[i]*\langle\sigma\text{v}\rangle[i, i]/H - 2*\bar{n}[i]*\langle\sigma\text{v}\rangle[j, j]/H - \bar{n}[j]*\langle\sigma\text{v}\rangle[i, j]/H - \tilde{\Gamma}_{SM}[i, j]/H - \Gamma[i]*\mathcal{N}^{th}[i]*m[i]/(H*R[i]*\bar{n}[i]))*\delta[i] - 3

#### Decoupled Regime

In [16]:
dNiDeCoupled = (dNiGeneral0+ dNiGeneral1).replace(d[i],0).replace(d[j],0).replace(D[i],n[i]-neq[i]).replace(D[j],n[j]-neq[j])
sp.collect(sp.simplify(dNiDeCoupled),coll)

(\bar{n}[i]**2/(H*n[i]) - n[i]/H)*\langle\sigma\text{v}\rangle[i, i] + (\bar{n}[i]*\bar{n}[j]/(H*n[i]) - n[j]/H)*\langle\sigma\text{v}\rangle[i, j] + (\mathcal{N}^{th}[i]*m[i]/(H*R[i]*n[i]) - m[i]/(H*R[i]))*\Gamma[i] + (\bar{n}[i]*n[j]/(H*\bar{n}[j]*n[i]) - 1/H)*\tilde{\Gamma}_{SM}[i, j] + (\bar{n}[i]**2*n[j]**2/(H*\bar{n}[j]**2*n[i]) - n[i]/H)*\langle\sigma\text{v}\rangle[j, j] + (-\Gamma[j]*\mathcal{N}^{th}[j, i]*m[j]/(H*R[j]*n[i]) + \Gamma[j]*m[j]*n[j]/(H*R[j]*n[i]))*\mathcal{B}^{eff}[j, i] - 3

## Energy Ratio Equations:

In [17]:
dRi = -3 *Pn[i]
dRi += Beff[j,i]*(w[j]*m[j]/H)*(sp.Rational(1,2) - R[i]/R[j])*(n[j]/n[i] - Nth[j,i]/n[i])
dRi = sp.collect(dRi,coll)

In [18]:
dRi

-3*P_{n}[i] + (-R[i]/R[j] + 1/2)*(-\mathcal{N}^{th}[j, i]/n[i] + n[j]/n[i])*\Gamma[j]*\mathcal{B}^{eff}[j, i]*m[j]/H

### Expansion of equations

In [19]:
#Replace n -> neq(1+delta) + Delta
#Coupled regime: Delta = 0, delta << 1 (expansion up to first order in delta)
#Non-coupled regime: delta = 0 (expansion up to order zero in delta)
#General scenario: expansion up to first order in delta and set delta_i*Delta_i = 0
dRiGeneral = dRi.subs({n[i] : neq[i]*(1+d[i])+D[i],n[j] : neq[j]*(1+d[j])+D[j]})
dRiGeneral = sp.collect(sp.expand(dRiGeneral).subs({d[i]*D[i] : 0, d[j]*D[j] : 0}),coll)

In [20]:
#Zero order term:
dRiGeneral0 = dRiGeneral.taylor_term(0,d[i]).taylor_term(0,d[j])
#First order term (1st order in delta_i with Delta_i = 0 + 1st order in delta_j with Delta_j=0)
dRiGeneral1 = dRiGeneral.taylor_term(1,d[i]).subs({D[i] : 0}).taylor_term(0,d[j])
dRiGeneral1 += dRiGeneral.taylor_term(0,d[i]).taylor_term(1,d[j]).subs({D[j] : 0})

In [21]:
exp0 = -3*Pn[i]
for term in coll:
    exp0 += sp.simplify(dRiGeneral0.coeff(term,1))*term
exp0

-3*P_{n}[i] + (2*(-\Delta[j] - \bar{n}[j] + \mathcal{N}^{th}[j, i])*R[i] + (\Delta[j] + \bar{n}[j] - \mathcal{N}^{th}[j, i])*R[j])*\Gamma[j]*\mathcal{B}^{eff}[j, i]*m[j]/(2*H*(\Delta[i] + \bar{n}[i])*R[j])

In [22]:
sp.simplify(exp0-dRiGeneral0)

0

In [23]:
exp1 = 0
for dterm in (d[i],d[j]):
    for term in coll:
        exp1 += sp.simplify(dRiGeneral1.coeff(dterm,1).coeff(term))*term*dterm
sp.simplify(sp.collect(exp1,(d[i],d[j])))
# exp1

((\Delta[i] + \bar{n}[i])*(\Delta[j] + \bar{n}[j] - \mathcal{N}^{th}[j, i])*\delta[i] - \bar{n}[i]*\bar{n}[j]*\delta[j])*(2*R[i] - R[j])*\Gamma[j]*\mathcal{B}^{eff}[j, i]*m[j]/(2*H*(\Delta[i] + \bar{n}[i])*R[j]*\bar{n}[i])

In [24]:
sp.simplify(exp1-dRiGeneral1)

0